# PersonaGPT Conversation with OpenVINO

This notebook shows Conversational Models with OpenVINO. We use the [PersonaGPT](https://arxiv.org/abs/2110.12949v1) model, which an open-domain conversational agent cpable of decoding _personalized_ and _controlled_ responses based on user input. It is built on the pretrained [DialoGPT-medium](https://github.com/microsoft/DialoGPT) model, following the [GPT-2](https://github.com/openai/gpt-2) architecture. 
PersonaGPT is fine-tuned on the [Persona-Chat](https://arxiv.org/pdf/1801.07243) dataset. The model is available from [HuggingFace](https://huggingface.co/af1tang/personaGPT). PersonaGPT displays a broad set of capabilities, including the ability to take on personas, where we prime the model with few facts and have it generate based upon that, it can also be used for creating a chatbot on a knowledge base.

The following image illustrates complete demo pipeline used for this scenario:

![image2](https://user-images.githubusercontent.com/95569637/226101538-e204aebd-a34f-4c8b-b90c-5363ba41c080.jpeg)

User Input is tokenized with eos_token concatenated in the end. Model input is tokenized text, which serves as initial condition for generation, then logits from model inference result should be obtained and token with the highest probability is selected using top-k sampling strategy and joined to input sequence. The procedure repeats until end of sequence token will be recived or specified maximum length is reached. After that, decoding token ids to text using tokenized should be applied.

The Generated response is added to the history with the eos_token at the end. Further User Input is added to it and agin passed into the model.

## The Model


In [96]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

pt_model = GPT2LMHeadModel.from_pretrained('af1tang/personaGPT')
tokenizer = GPT2Tokenizer.from_pretrained('af1tang/personaGPT')

## Convert PersonaGPT to OpenVINO IR

![conversion_pipeline](https://user-images.githubusercontent.com/29454499/211261803-784d4791-15cb-4aea-8795-0969dfbb8291.png)

For starting work with GPT-Neo model using OpenVINO, model should be converted to OpenVINO Intermediate Represenation (IR) format. HuggingFace provided gpt-neo model is PyTorch model, which supported in OpenVINO via conversion to ONNX. We will use HuggingFace transformers library capabilities for export model to ONNX. `transformers.onnx.export` accept preprocessing function for input sample generation (tokenizer in our case), instance of model, ONNX export configuration, ONNX opset version for export and output path. More information about transformers export to ONNX can be found in HuggingFace [documentation](https://huggingface.co/docs/transformers/serialization).

While ONNX models are directly supported by OpenVINO runtime, it can be useful to convert them to IR format to take advantage of OpenVINO optimization tools and features.
`mo.convert_model` python function can be used for converting model using [OpenVINO Model Optimizer](https://docs.openvino.ai/latest/openvino_docs_MO_DG_Python_API.html). The function returns instance of OpenVINO Model class, which is ready to use in Python interface but can also be serialized to OpenVINO IR format for future execution using `openvino.runtime.serialize`. In our case, `compress_to_fp16` parameter is enabled for compression model weights to fp16 precision and also specified dynamic input shapes with possible shape range (from 1 token to maximum length defined in our processing function) for optimization of memory consumption.

In [91]:
from pathlib import Path
from openvino.runtime import serialize
from openvino.tools import mo
from transformers.onnx import export, FeaturesManager


# define path for saving onnx model
onnx_path = Path("model/personaGPT.onnx")
onnx_path.parent.mkdir(exist_ok=True)

# define path for saving openvino model
model_path = onnx_path.with_suffix(".xml")

# get model onnx config function for output feature format casual-lm
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(pt_model, feature='causal-lm')

# fill onnx config based on pytorch model config
onnx_config = model_onnx_config(pt_model.config)

# convert model to onnx
onnx_inputs, onnx_outputs = export(preprocessor=tokenizer, model=pt_model, config=onnx_config, opset=onnx_config.default_onnx_opset, output=onnx_path)

# convert model to openvino
ov_model = mo.convert_model(onnx_path, compress_to_fp16=True, input="input_ids[1,1..1000],attention_mask[1,1..1000]")

# serialize openvino model
serialize(ov_model, str(model_path))

c:\Users\Asus\miniconda3\envs\deep\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:794: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if batch_size <= 0:


### Load the model

We start by building an OpenVINO Core object. Then we read the network architecture and model weights from the .xml and .bin files, respectively. Finally, we compile the model for the desired device. Because we use the dynamic shapes feature, which is only available on CPU, we must use `CPU` for the device. Dynamic shapes support on GPU is coming soon.

Since the text recognition model has a dynamic input shape, you cannot directly switch device to `GPU` for inference on integrated or discrete Intel GPUs. In order to run inference on iGPU or dGPU with this model, you will need to resize the inputs to this model to use a fixed size and then try running the inference on `GPU` device.

In [92]:
from openvino.runtime import Core

# initialize openvino core
core = Core()

# read the model and corresponding weights from file
model = core.read_model(model_path)

# compile the model for CPU devices
compiled_model = core.compile_model(model=model, device_name="CPU")

# get output tensors
output_key = compiled_model.output(0)

Input keys are the names of the input nodes and output keys contain names of the output nodes of the network. In the case of GPT-2, we have `batch size` and `sequence length` as inputs and `batch size`, `sequence length` and `vocab size` as outputs.

## Pre-Processing

NLP models often take a list of tokens as a standard input. A token is a single word mapped to an integer. To provide the proper input, we use a vocabulary file to handle the mapping. So first let's load the vocabulary file.

## Define tokenization

In [5]:
from typing import List, Tuple


# this function converts text to tokens
def tokenize(text: str) -> Tuple[List[int], List[int]]:
    """
    tokenize input text using GPT2 tokenizer

    Parameters:
      text, str - input text
    Returns:
      input_ids - np.array with input token ids
      attention_mask - np.array with 0 in place, where should be padding and 1 for places where original tokens are located, represents attention mask for model
    """

    inputs = tokenizer(text, return_tensors="np")
    return inputs["input_ids"], inputs["attention_mask"]

`eos_token` is special token, which means that generation is finished. We store the index of this token in order to use this index as padding at later stage.

In [6]:
eos_token_id = tokenizer.eos_token_id
eos_token = tokenizer.decode(eos_token_id)

### Define Softmax layer
A softmax function is used to convert top-k logits into a probability distribution. 

In [7]:
import numpy as np


def softmax(x: np.array) -> np.array:
    """
    Computes the Softmax
    """
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    summation = e_x.sum(axis=-1, keepdims=True)
    return e_x / summation

### Set the minimum sequence length  
If the minimum sequence length is not reached, the following code will reduce the probability of the `eos` token occurring. This continues the process of generating the next words.

In [8]:
def process_logits(cur_length: int, scores: np.array, eos_token_id: int, min_length: int = 0) -> np.array:
    """
    reduce probability for padded indicies

    Parameters:
      cur_length - current length of input sequence
      scores - model output logits
      eos_token_id - index of end of string token in model vocab
      min_length - minimum length for appling postprocessing
    """
    if cur_length < min_length:
        scores[:, eos_token_id] = -float("inf")
    return scores

### Top-K sampling
In Top-K sampling, we filter the K most likely next words and redistribute the probability mass among only those K next words. 

In [9]:
def get_top_k_logits(scores: np.array, top_k: int) -> np.array:
    """
    perform top-k sampling

    Parameters:
      scores - model output logits
      top_k - number of elements with highest probability to select
    """
    filter_value = -float("inf")
    top_k = min(max(top_k, 1), scores.shape[-1])
    top_k_scores = -np.sort(-scores)[:, :top_k]
    indices_to_remove = scores < np.min(top_k_scores)
    filtred_scores = np.ma.array(scores, mask=indices_to_remove,
                                 fill_value=filter_value).filled()
    return filtred_scores

### Main Processing Function
Generating the predicted sequence.

In [23]:
def generate_sequence(input_ids: List[int], attention_mask: List[int], max_sequence_length: int = 1000,
                      eos_token_id: int = eos_token_id, dynamic_shapes: bool = False) -> List[int]:
    """
    text prediction cycle.

    Parameters:
      input_ids: tokenized input ids for model
      attention_mask: attention mask for model
      max_sequence_length: maximum sequence length for stop iteration
      eos_token_id: end of sequence index from vocab
      dynamic_shapes: use dynamic shapes for inference or pad model input to max_sequece_length
    Returns:
      predicted token ids sequence
    """
    while True:
        cur_input_len = len(input_ids[0])
        if not dynamic_shapes:
            pad_len = max_sequence_length - cur_input_len
            model_input_ids = np.concatenate((input_ids, [[eos_token_id] * pad_len]), axis=-1)
            model_input_attention_mask = np.concatenate((attention_mask, [[0] * pad_len]), axis=-1)
        else:
            model_input_ids = input_ids
            model_input_attention_mask = attention_mask
        outputs = compiled_model({"input_ids": model_input_ids, "attention_mask": model_input_attention_mask})[output_key]
        next_token_logits = outputs[:, cur_input_len - 1, :]
        # pre-process distribution
        next_token_scores = process_logits(cur_input_len,
                                           next_token_logits, eos_token_id)
        top_k = 20
        next_token_scores = get_top_k_logits(next_token_scores, top_k)
        # get next token id
        probs = softmax(next_token_scores)
        next_tokens = np.random.choice(probs.shape[-1], 1,
                                       p=probs[0], replace=True)
        # break the loop if max length or end of text token is reached
        # print(next_tokens[0])
        # print(cur_input_len,max_sequence_length)
        if cur_input_len == max_sequence_length or next_tokens[0] == eos_token_id:
            break
        else:
            input_ids = np.concatenate((input_ids, [next_tokens]), axis=-1)
            attention_mask = np.concatenate((attention_mask, [[1] * len(next_tokens)]), axis=-1)
    return input_ids

## Converse Function
Wrapper on generate sequence function to support conversation

In [ ]:
def converse(input: str, history: List[int], eos_token: str = eos_token,
             eos_token_id: int = eos_token_id) -> Tuple[str, List[int]]:
    """
    Converse with the Model.

    Parameters:
      input: Text input given by the User
      history: Chat History, ids of tokens of chat occured so far
      eos_token: end of sequence string
      eos_token_id: end of sequence index from vocab
    Returns:
      response: Text Response generated by the model
      history: Chat History, Ids of the tokens of chat occured so far,including the tokens of generated response
    """

    print("Person:", input)

    # Get Input Ids of the User Input
    new_user_input_ids, _ = tokenize(input + eos_token)

    # append the new user input tokens to the chat history, if history exists
    if len(history) == 0:
        bot_input_ids = new_user_input_ids
    else:
        bot_input_ids = np.concatenate([history, new_user_input_ids[0]])
        bot_input_ids = np.expand_dims(bot_input_ids, axis=0)

    # Create Attention Mask
    bot_attention_mask = np.ones_like(bot_input_ids)

    # Generate Response from the model
    history = generate_sequence(bot_input_ids, bot_attention_mask)

    # Add the eos_token to mark end of sequence
    history = np.append(history[0], eos_token_id)

    # convert the tokens to text, and then split the responses into lines and retrieve the response from the Model
    response = ''.join(tokenizer.batch_decode(history)).split(eos_token)[-2]
    return response, history

## Chat

In [99]:
# Variables for conversation
history = []
user_prompt = None
pre_written_prompts = ["Hi,How are you?", "What are you doing?", "I like to dance,do you?", "Can you recommend me some books?"]
# Number of responses generated by model
n_prompts = 10
for i in range(n_prompts):
    # Uncomment for taking User Input
    # user_prompt = input()
    if not user_prompt:
        user_prompt = pre_written_prompts[i % len(pre_written_prompts)]
    response, history = converse(user_prompt, history)
    print("PersonaGPT:", response)
    user_prompt = None

Person: Hi, How are you?
PersonaGPT: good how are you
Person: I am good too! What are your hobbies?
PersonaGPT: i like to play sports
Person: Really! I like to play sports too. Which sport do you play?
PersonaGPT: soccer since i'm a pro
Person: Since when have you been playing soccer?
PersonaGPT: since highschool now how about you
Person: I like to play badminton
PersonaGPT: that is cool i don't play sports
Person: What are you doing?
PersonaGPT: studying and eating dinner
Person: That's nice,How?
PersonaGPT: i work really hard to get my grades where i want them
Person: Why do you want grades?
PersonaGPT: for the
Person: Do you like school?
PersonaGPT: yeah i do but i also like to play sports
Person: Great!
PersonaGPT: what else do you like to do
